In [1]:
import tensorflow as tf
import mnist_inference
import os
import pandas as pd
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
BATCH_SIZE = 200 
LEARNING_RATE_BASE = 0.99
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.03
TRAINING_STEPS = 1500

In [3]:
MOVING_AVERAGE_DECAY = 0.49
MODEL_SAVE_PATH = "../model/{}".format(pd.Timestamp.now())[:19]
MODEL_NAME = "tianchi_model"
TENSORBOARD_LOG = 'tensor_board'

In [4]:
def train(trainX, trainY, testX, testY):
    dataSize = len(trainY)
    
    with tf.device('/gpu:0'):
        # 定义输入输出placeholder。
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')

        regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
        y = mnist_inference.inference(x, regularizer)
        global_step = tf.Variable(0, trainable=False)
    
        # 定义损失函数、学习率、滑动平均操作以及训练过程。
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
        #cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
        #cross_entropy_mean = tf.reduce_mean(cross_entropy)
        beginLoss = tf.sqrt(tf.reduce_mean(tf.pow(tf.subtract(y, y_), 2)))
        #loss = beginLoss + tf.add_n(tf.get_collection('losses'))
        loss = beginLoss
        learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            dataSize / BATCH_SIZE, LEARNING_RATE_DECAY,
            staircase=True)
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        with tf.control_dependencies([train_step, variables_averages_op]):
            train_op = tf.no_op(name='train')
        
    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    with tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True)) as sess:
        tf.global_variables_initializer().run()
        
        for i in range(TRAINING_STEPS):
            for start in range(0, len(trainX)-BATCH_SIZE, BATCH_SIZE):
                end = start + BATCH_SIZE
                if end <=len(trainX):
                    x_batch, y_batch = trainX[start: end], trainY[start: end]
                else: 
                    x_batch, y_batch = trainX[start: ], trainY[start: ]
                _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: x_batch, y_: y_batch})
                if start % len(trainX) == 0:
                    print("After", step," training step(s), loss on training batch is ", loss_value)
            if i % 5 == 0:
                test_loss = sess.run(loss, feed_dict={x:test_x, y_: test_y})
                print("rang", i, "current test loss value on test set is ", test_loss)
        saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

In [5]:
df = pd.read_csv('../CleanData/Max_trainDataAfterClean.csv')
# random shift the df
df = df.sample(frac=1).reset_index(drop=True)

normalizeColumns = ['compartment','TR','displacement','price_level','power','level_id',
                    'cylinder_number','engine_torque','car_length','car_height','car_width','total_quality','equipment_quality',
                    'rated_passenger','wheelbase','front_track','rear_track']
leftDf = df.drop(normalizeColumns, axis =1 ).drop(['sale_quantity'], axis = 1)

normalizeDf = df[normalizeColumns]
normalizeDf = (normalizeDf-normalizeDf.min())/(normalizeDf.max()-normalizeDf.min())
inputDf = pd.concat([leftDf, normalizeDf], axis = 1)
inputX = inputDf.values
resultArray = df['sale_quantity'].values
inputY = resultArray.reshape((len(resultArray),1))

train_x, test_x, train_y, test_y = train_test_split(inputX, inputY, test_size=0.2, random_state=42)

In [6]:
train(train_x, train_y, test_x, test_y)

After 1  training step(s), loss on training batch is  438.711
rang 0 current test loss value on test set is  427.112
After 24  training step(s), loss on training batch is  432.96
After 47  training step(s), loss on training batch is  331.983
After 70  training step(s), loss on training batch is  422.648
After 93  training step(s), loss on training batch is  417.81
After 116  training step(s), loss on training batch is  412.112
rang 5 current test loss value on test set is  400.942
After 139  training step(s), loss on training batch is  406.443
After 162  training step(s), loss on training batch is  397.284
After 185  training step(s), loss on training batch is  397.299
After 208  training step(s), loss on training batch is  393.528
After 231  training step(s), loss on training batch is  388.956
rang 10 current test loss value on test set is  380.3
After 254  training step(s), loss on training batch is  385.393
After 277  training step(s), loss on training batch is  381.741
After 300  t

After 2531  training step(s), loss on training batch is  270.173
rang 110 current test loss value on test set is  273.607
After 2554  training step(s), loss on training batch is  270.003
After 2577  training step(s), loss on training batch is  266.517
After 2600  training step(s), loss on training batch is  266.158
After 2623  training step(s), loss on training batch is  265.146
After 2646  training step(s), loss on training batch is  267.243
rang 115 current test loss value on test set is  265.664
After 2669  training step(s), loss on training batch is  264.107
After 2692  training step(s), loss on training batch is  264.264
After 2715  training step(s), loss on training batch is  263.222
After 2738  training step(s), loss on training batch is  263.58
After 2761  training step(s), loss on training batch is  262.391
rang 120 current test loss value on test set is  264.084
After 2784  training step(s), loss on training batch is  261.968
After 2807  training step(s), loss on training bat

After 4992  training step(s), loss on training batch is  205.881
After 5015  training step(s), loss on training batch is  218.08
After 5038  training step(s), loss on training batch is  231.701
After 5061  training step(s), loss on training batch is  238.088
rang 220 current test loss value on test set is  201.486
After 5084  training step(s), loss on training batch is  182.559
After 5107  training step(s), loss on training batch is  281.708
After 5130  training step(s), loss on training batch is  184.052
After 5153  training step(s), loss on training batch is  172.294
After 5176  training step(s), loss on training batch is  178.855
rang 225 current test loss value on test set is  307.286
After 5199  training step(s), loss on training batch is  303.54
After 5222  training step(s), loss on training batch is  284.876
After 5245  training step(s), loss on training batch is  282.705
After 5268  training step(s), loss on training batch is  263.256
After 5291  training step(s), loss on train

After 7476  training step(s), loss on training batch is  133.193
rang 325 current test loss value on test set is  147.201
After 7499  training step(s), loss on training batch is  133.85
After 7522  training step(s), loss on training batch is  132.25
After 7545  training step(s), loss on training batch is  132.498
After 7568  training step(s), loss on training batch is  132.878
After 7591  training step(s), loss on training batch is  132.269
rang 330 current test loss value on test set is  146.92
After 7614  training step(s), loss on training batch is  132.826
After 7637  training step(s), loss on training batch is  132.302
After 7660  training step(s), loss on training batch is  132.805
After 7683  training step(s), loss on training batch is  133.119
After 7706  training step(s), loss on training batch is  132.804
rang 335 current test loss value on test set is  146.792
After 7729  training step(s), loss on training batch is  132.655
After 7752  training step(s), loss on training batch

After 9983  training step(s), loss on training batch is  124.371
After 10006  training step(s), loss on training batch is  124.182
rang 435 current test loss value on test set is  144.101
After 10029  training step(s), loss on training batch is  124.188
After 10052  training step(s), loss on training batch is  123.987
After 10075  training step(s), loss on training batch is  123.979
After 10098  training step(s), loss on training batch is  123.966
After 10121  training step(s), loss on training batch is  123.722
rang 440 current test loss value on test set is  143.948
After 10144  training step(s), loss on training batch is  123.652
After 10167  training step(s), loss on training batch is  123.531
After 10190  training step(s), loss on training batch is  123.517
After 10213  training step(s), loss on training batch is  123.445
After 10236  training step(s), loss on training batch is  123.392
rang 445 current test loss value on test set is  143.836
After 10259  training step(s), loss on

...

rang 970 current test loss value on test set is  142.763
After 22334  training step(s), loss on training batch is  115.993
After 22357  training step(s), loss on training batch is  115.99
After 22380  training step(s), loss on training batch is  115.992
After 22403  training step(s), loss on training batch is  115.991
After 22426  training step(s), loss on training batch is  115.99
rang 975 current test loss value on test set is  142.763
After 22449  training step(s), loss on training batch is  115.991
After 22472  training step(s), loss on training batch is  115.987
After 22495  training step(s), loss on training batch is  115.987
After 22518  training step(s), loss on training batch is  115.989
After 22541  training step(s), loss on training batch is  115.986
rang 980 current test loss value on test set is  142.762
After 22564  training step(s), loss on training batch is  115.986
After 22587  training step(s), loss on training batch is  115.987
After 22610  training step(s), loss on 

After 24818  training step(s), loss on training batch is  115.956
After 24841  training step(s), loss on training batch is  115.956
rang 1080 current test loss value on test set is  142.758
After 24864  training step(s), loss on training batch is  115.957
After 24887  training step(s), loss on training batch is  115.955
After 24910  training step(s), loss on training batch is  115.956
After 24933  training step(s), loss on training batch is  115.955
After 24956  training step(s), loss on training batch is  115.955
rang 1085 current test loss value on test set is  142.758
After 24979  training step(s), loss on training batch is  115.955
After 25002  training step(s), loss on training batch is  115.955
After 25025  training step(s), loss on training batch is  115.955
After 25048  training step(s), loss on training batch is  115.954
After 25071  training step(s), loss on training batch is  115.954
rang 1090 current test loss value on test set is  142.758
After 25094  training step(s), los

After 27302  training step(s), loss on training batch is  115.945
After 27325  training step(s), loss on training batch is  115.945
After 27348  training step(s), loss on training batch is  115.945
After 27371  training step(s), loss on training batch is  115.945
rang 1190 current test loss value on test set is  142.757
After 27394  training step(s), loss on training batch is  115.945
After 27417  training step(s), loss on training batch is  115.945
After 27440  training step(s), loss on training batch is  115.945
After 27463  training step(s), loss on training batch is  115.945
After 27486  training step(s), loss on training batch is  115.945
rang 1195 current test loss value on test set is  142.757
After 27509  training step(s), loss on training batch is  115.945
After 27532  training step(s), loss on training batch is  115.945
After 27555  training step(s), loss on training batch is  115.945
After 27578  training step(s), loss on training batch is  115.945
After 27601  training step

After 29740  training step(s), loss on training batch is  115.941
After 29763  training step(s), loss on training batch is  115.941
After 29786  training step(s), loss on training batch is  115.941
rang 1295 current test loss value on test set is  142.756
After 29809  training step(s), loss on training batch is  115.941
After 29832  training step(s), loss on training batch is  115.941
After 29855  training step(s), loss on training batch is  115.941
After 29878  training step(s), loss on training batch is  115.941
After 29901  training step(s), loss on training batch is  115.941
rang 1300 current test loss value on test set is  142.756
After 29924  training step(s), loss on training batch is  115.941
After 29947  training step(s), loss on training batch is  115.941
After 29970  training step(s), loss on training batch is  115.941
After 29993  training step(s), loss on training batch is  115.941
After 30016  training step(s), loss on training batch is  115.941
rang 1305 current test los

After 32178  training step(s), loss on training batch is  115.938
After 32201  training step(s), loss on training batch is  115.938
rang 1400 current test loss value on test set is  142.756
After 32224  training step(s), loss on training batch is  115.938
After 32247  training step(s), loss on training batch is  115.938
After 32270  training step(s), loss on training batch is  115.938
After 32293  training step(s), loss on training batch is  115.938
After 32316  training step(s), loss on training batch is  115.938
rang 1405 current test loss value on test set is  142.756
After 32339  training step(s), loss on training batch is  115.938
After 32362  training step(s), loss on training batch is  115.938
After 32385  training step(s), loss on training batch is  115.938
After 32408  training step(s), loss on training batch is  115.938
After 32431  training step(s), loss on training batch is  115.938
rang 1410 current test loss value on test set is  142.756
After 32454  training step(s), los